<img src=https://static.wixstatic.com/media/f6bd9a_6a972d9b31324653bf198c38ce94339e~mv2.png>

# Carrusel de RDBMS
## Ejercicios de SQL resueltos

En este cuaderno resuelvo los ejercicios del 61 al 70 del documento: 'Desarrollo de Aplicaciones en Entornos de Cuarta Generación y Herramientas CASE: Ejercicios de SQL.'

Para ello utilizo una version de la base de datos -que nombre "practicas_hospital"- modificada para ser compatible en esta ocasion, para **PostgreSQL**, siguiendo la estructura del libro.


Tanto el diagrama de **Entidad-Relacion**, como el **Script** para cargar la base de datos (que permite, se pueda replicar el ejercicio) se encuentran disponibles en el mismo repositorio.

#### Celda para configurar el entorno

In [ ]:
# !pip install psycopg2
# !pip install pandas
# !pip install sqlalchemy

#### Importamos librerias y se realiza la conexion utilizando un archivo json con la configuracion local de la base de datos

Si se desea replicar lo propio usando el script y este cuaderno, basta con crear el archivo json con el par de valores que contengan su informacion local y cargarlo al directorio.

host:

user:

password:

database:

In [1]:
import json
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd

# Cargar las credenciales desde el archivo JSON
with open('configPostgres.json', 'r') as f:
    credenciales = json.load(f)

# Extraer las credenciales
host = credenciales['host']
database = credenciales['database']
user = credenciales['user']
password = credenciales['password']

# Crear la conexión
engine = create_engine(f'postgresql://{user}:{password}@{host}/{database}')


#### Realizamos una prueba y se Inspecciona la base de datos "practicas_hospital"

In [2]:
# Definir consulta en SQL compatible con MySQL
try:
    QUERY = """
    SELECT tablename 
    FROM pg_catalog.pg_tables
    WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';
    """
# Ejecutar la consulta y obtener los resultados en un DataFrame
    df = pd.read_sql_query(text(QUERY), engine)
# Mostrar los primeros resultados
    print("La conexión se ha realizado con éxito.\n")
    print(df)
except Exception as e:
    print("Hubo un error al realizar la conexión:", e)

La conexión se ha realizado con éxito.

      tablename
0      hospital
1          sala
2     plantilla
3     ocupacion
4        doctor
5       enfermo
6      empleado
7  departamento


#### Inspeccion a las tablas de la base de datos "practicas_hospital"
#### Con la finalidad de observar su estructura y tipo de datos

In [3]:
for valor in df['tablename']:
    QUERY = f"SELECT column_name, data_type, character_maximum_length, is_nullable FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = '{valor}'" 
    df1 = pd.read_sql_query(QUERY, engine)
    print(f'El contenido de la tabla {valor}\n')
    print(df1)
    print()

El contenido de la tabla hospital

    column_name          data_type  character_maximum_length is_nullable
0  hospital_cod            integer                       NaN          NO
1        nombre  character varying                      10.0         YES
2     direccion  character varying                      20.0         YES
3      telefono  character varying                       8.0         YES
4      num_cama            integer                       NaN         YES

El contenido de la tabla sala

    column_name          data_type  character_maximum_length is_nullable
0       sala_id            integer                       NaN          NO
1  hospital_cod            integer                       NaN          NO
2      sala_cod            integer                       NaN          NO
3        nombre  character varying                      20.0         YES
4      num_cama            integer                       NaN         YES

El contenido de la tabla plantilla

    column_name     

### Resolucion de los ejercicios numerados del 61 al 70

##### 61. Esta inserción falla. ¿Por qué?.

 INSERT INTO PLANTILLA (empleado_no, apellido) VALUES (1234,'Garcia J)

In [4]:
try:
    with engine.begin() as connection:
        QUERY = """
        INSERT INTO PLANTILLA (empleado_no, apellido) values (1234,'Garcia J)
        """
        result = connection.execute(text(QUERY))
    print(f' se ha modificado {result.rowcount} fila(s)')
except Exception as e:
    print("Hubo un error:\n\n", e, '\n')

Hubo un error:

 (psycopg2.errors.SyntaxError) una cadena de caracteres entre comillas está inconclusa en o cerca de «'Garcia J)
        »
LINE 2: ...TO PLANTILLA (empleado_no, apellido) values (1234,'Garcia J)
                                                             ^

[SQL: 
        INSERT INTO PLANTILLA (empleado_no, apellido) values (1234,'Garcia J)
        ]
(Background on this error at: https://sqlalche.me/e/20/f405) 



##### 62. En esta inserción no se contemplan todos los campos de la tabla, ¿Falla la inserción?.


INSERT INTO PLANTILLA (hospital_cod, sala_cod, empleado_no, apellido) VALUES (2,22,1234,'Garcia J');

In [5]:
with engine.begin() as connection:
    QUERY = """
    INSERT INTO PLANTILLA (hospital_cod, sala_cod, empleado_no, apellido) VALUES (2,22,1234,'Garcia J');
    """
    result = connection.execute(text(QUERY))
print(f' se ha modificado {result.rowcount} fila(s)')
print('Como se vio en la descripcion de las tablas, hay campos que no son especificados como NOT NULL')

 se ha modificado 1 fila(s)
Como se vio en la descripcion de las tablas, hay campos que no son especificados como NOT NULL


In [6]:
QUERY = """
SELECT * FROM PLANTILLA;
"""
# Ejecutar la consulta y obtener los resultados en un DataFrame
df = pd.read_sql_query(text(QUERY), engine)
df

,hospital_cod,sala_cod,empleado_no,apellido,funcion,turno,salario
0,13,6,3754,Diaz B.,Enfermera,T,2262000.0
1,13,6,3106,Hernandez J.,Enfermero,T,2755000.0
2,18,4,6357,Karplus W.,Interno,T,3379000.0
3,22,6,1009,Higueras D.,Enfermera,T,2005000.0
4,22,6,8422,Bocina G.,Enfermero,M,1638000.0
5,22,2,9901,NuÑez C.,Interno,M,2210000.0
6,22,1,6065,Rivera G.,Enfermera,N,1626000.0
7,22,1,7379,Carlos R.,Enfermera,T,2119000.0
8,45,4,1280,Amigo R.,Interno,N,2210000.0
9,45,1,8526,Frank H.,Enfermera,T,2522000.0


#### ACTUALIZACIONES.

##### 63. Cambiar al paciente (tabla ENFERMO) número 74835 la dirección a Alcala 411.

In [7]:
with engine.begin() as connection:
    QUERY = """
    UPDATE ENFERMO
    SET DIRECCION = 'Alcala 411'
    WHERE INSCRIPCION = 74835;
    """
    result = connection.execute(text(QUERY))
print(f' se ha modificado {result.rowcount} fila(s)')

 se ha modificado 1 fila(s)


In [8]:
QUERY = """
SELECT * FROM ENFERMO;
"""
# Ejecutar la consulta y obtener los resultados en un DataFrame
df = pd.read_sql_query(text(QUERY), engine)
df

,enfermo_no,inscripcion,apellido,direccion,fecha_nac,s,nss
0,1,10995,Laguia M.,Recoletos 50,1967-06-23,M,280862482
1,2,18004,Serrano V.,Alcala 12,1960-05-21,F,284991452
2,3,14024,Fernandez M,Recoletos 500,1977-06-23,F,321790059
3,4,36658,Domin S.,Mayor 71,1942-01-01,M,160657471
4,5,38702,Neal R.,Orense 11,1940-06-18,F,380010217
5,6,39217,Cervantes M.,Peron 38,1952-02-29,M,440294390
6,7,59076,Miller G.,Lopez de Hoyos 2,1945-09-16,F,311969044
7,8,63827,Ruiz P.,Esquerdo 103,1980-12-26,M,100973253
8,9,64823,Fraser A.,Soto 3,1980-07-10,F,285201776
9,10,74835,Benitez E.,Alcala 411,1957-10-05,M,154811767


##### 64. Poner todas las direcciones de la tabla ENFERMO a null.

In [9]:
with engine.begin() as connection:
    QUERY = """
    UPDATE ENFERMO
    SET DIRECCION = NULL;
    """
    result = connection.execute(text(QUERY))
print(f' se ha modificado {result.rowcount} fila(s)')

 se ha modificado 10 fila(s)


In [10]:
QUERY = """
SELECT * FROM ENFERMO;
"""
# Ejecutar la consulta y obtener los resultados en un DataFrame
df = pd.read_sql_query(text(QUERY), engine)
df

,enfermo_no,inscripcion,apellido,direccion,fecha_nac,s,nss
0,1,10995,Laguia M.,None,1967-06-23,M,280862482
1,2,18004,Serrano V.,None,1960-05-21,F,284991452
2,3,14024,Fernandez M,None,1977-06-23,F,321790059
3,4,36658,Domin S.,None,1942-01-01,M,160657471
4,5,38702,Neal R.,None,1940-06-18,F,380010217
5,6,39217,Cervantes M.,None,1952-02-29,M,440294390
6,7,59076,Miller G.,None,1945-09-16,F,311969044
7,8,63827,Ruiz P.,None,1980-12-26,M,100973253
8,9,64823,Fraser A.,None,1980-07-10,F,285201776
9,10,74835,Benitez E.,None,1957-10-05,M,154811767


##### 65. Igualar la dirección y fecha de nacimiento del paciente 10995 a los valores de las columnas correspondientes almacenadas para el paciente 14024.

In [11]:
with engine.begin() as connection:
    QUERY = """
    UPDATE ENFERMO
    SET (DIRECCION, FECHA_NAC) = (SELECT DIRECCION, FECHA_NAC
								FROM ENFERMO
								WHERE INSCRIPCION = 10995)
    WHERE INSCRIPCION = 14024;
    """
    result = connection.execute(text(QUERY))
print(f' se ha modificado {result.rowcount} fila(s)')

 se ha modificado 1 fila(s)


In [12]:
QUERY = """
SELECT * FROM ENFERMO;
"""
# Ejecutar la consulta y obtener los resultados en un DataFrame
df = pd.read_sql_query(text(QUERY), engine)
df

,enfermo_no,inscripcion,apellido,direccion,fecha_nac,s,nss
0,1,10995,Laguia M.,None,1967-06-23,M,280862482
1,2,18004,Serrano V.,None,1960-05-21,F,284991452
2,4,36658,Domin S.,None,1942-01-01,M,160657471
3,5,38702,Neal R.,None,1940-06-18,F,380010217
4,6,39217,Cervantes M.,None,1952-02-29,M,440294390
5,7,59076,Miller G.,None,1945-09-16,F,311969044
6,8,63827,Ruiz P.,None,1980-12-26,M,100973253
7,9,64823,Fraser A.,None,1980-07-10,F,285201776
8,10,74835,Benitez E.,None,1957-10-05,M,154811767
9,3,14024,Fernandez M,None,1967-06-23,F,321790059


##### 66. En todos los hospitales del país se ha recibido un aumento del presupuesto, por lo que se incrementará el número de camas disponibles en un 10%. ¿Como se haría en SQL?.

In [13]:
with engine.begin() as connection:
    QUERY = """
    UPDATE HOSPITAL
    SET NUM_CAMA = ROUND(NUM_CAMA * 1.1);
    """
    result = connection.execute(text(QUERY))
print(f' se ha modificado {result.rowcount} fila(s)')

 se ha modificado 4 fila(s)


In [14]:
QUERY = """
SELECT * FROM HOSPITAL;
"""
# Ejecutar la consulta y obtener los resultados en un DataFrame
df = pd.read_sql_query(text(QUERY), engine)
df

,hospital_cod,nombre,direccion,telefono,num_cama
0,13,Provincial,O Donell 50,964-4264,552
1,18,General,Atocha s/n,595-3111,1086
2,22,La Paz,Castellana 1000,923-5411,453
3,45,San Carlos,Ciudad Universitaria,597-1500,930


#### CREACIÓN DE TABLAS

##### 67.-Ejercicio a comentar su solución. Crear una tabla llamada hospital22 a partir de la tabla HOSPITAL. 

In [15]:
with engine.begin() as connection:
    # Se crea una copia desde la tabla hospital
    QUERY = """
    CREATE TABLE Hospital22 AS TABLE HOSPITAL WITH NO DATA;
    """
    result = connection.execute(text(QUERY))
    # Se agrega la primary key y si las hubiera, las restricciones
    QUERY = """
    ALTER TABLE Hospital22 ADD PRIMARY KEY (HOSPITAL_COD);
    """
    # Se comprueba que la tabla ha sido creada
    result = connection.execute(text(QUERY))
    QUERY = """
    SELECT * FROM Hospital22;
    """
    result = connection.execute(text(QUERY))
print(f'La nueva tabla tiene {result.rowcount} fila(s)')


La nueva tabla tiene 0 fila(s)


##### Se ejecuta el codigo que usamos arriba para revisar las tablas

In [16]:
QUERY = f"SELECT column_name, data_type, character_maximum_length, is_nullable FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'hospital22'" 
df1 = pd.read_sql_query(QUERY, engine)
print(f'El contenido de la tabla hospital22\n')
print(df1)
print()

El contenido de la tabla hospital22

    column_name          data_type  character_maximum_length is_nullable
0  hospital_cod            integer                       NaN          NO
1        nombre  character varying                      10.0         YES
2     direccion  character varying                      20.0         YES
3      telefono  character varying                       8.0         YES
4      num_cama            integer                       NaN         YES



##### 68.- Rellenar la tabla HOSPITALES22 con las filas de HOSPITAL. Da esto algún ¿problema? ¿Por que?
Repetir la operación. ¿DA PROBLEMAS?


In [17]:
# 1a vez
with engine.begin() as connection:
    QUERY = """
    INSERT INTO Hospital22
    SELECT * FROM HOSPITAL;
    """
    result = connection.execute(text(QUERY))
print(f' se ha modificado {result.rowcount} fila(s)')

 se ha modificado 4 fila(s)


In [18]:
QUERY = """
SELECT * FROM HOSPITAL22;
"""
# Ejecutar la consulta y obtener los resultados en un DataFrame
df = pd.read_sql_query(text(QUERY), engine)
df

,hospital_cod,nombre,direccion,telefono,num_cama
0,13,Provincial,O Donell 50,964-4264,552
1,18,General,Atocha s/n,595-3111,1086
2,22,La Paz,Castellana 1000,923-5411,453
3,45,San Carlos,Ciudad Universitaria,597-1500,930


Al estar la tabla vacia no hay ningun problema

In [19]:
# 2a vez
try:
    with engine.begin() as connection:
        QUERY = """
        INSERT INTO Hospital22
        SELECT * FROM HOSPITAL;
        """
        result = connection.execute(text(QUERY))
    print(f' se ha modificado {result.rowcount} fila(s)')
except Exception as e:
    print("Hubo un error:\n\n", e, '\n')

Hubo un error:

 (psycopg2.errors.UniqueViolation) llave duplicada viola restricción de unicidad «hospital22_pkey»
DETAIL:  Ya existe la llave (hospital_cod)=(13).

[SQL: 
        INSERT INTO Hospital22
        SELECT * FROM HOSPITAL;
        ]
(Background on this error at: https://sqlalche.me/e/20/gkpj) 



La restriccion de la primary key no permite duplicar las llaves. En el Script se le definio a cada tabla una primary key como parte de la normalizacion.

##### 69. Crearse una tabla llamada VARONES con la misma estructura que la tabla enfermo.

In [20]:
with engine.begin() as connection:
    # Se crea una copia desde la tabla ENFERMO
    QUERY = """
    CREATE TABLE VARONES AS TABLE ENFERMO WITH NO DATA;
    """
    result = connection.execute(text(QUERY))
    # Se agrega la primary key y si las hubiera, las restricciones
    QUERY = """
    ALTER TABLE VARONES ADD PRIMARY KEY (ENFERMO_NO);
    """
    # Se comprueba que la tabla ha sido creada
    result = connection.execute(text(QUERY))
    QUERY = """
    SELECT * FROM VARONES;
    """
    result = connection.execute(text(QUERY))
print(f'La nueva tabla tiene {result.rowcount} fila(s)')


La nueva tabla tiene 0 fila(s)


In [21]:
QUERY = f"SELECT column_name, data_type, character_maximum_length, is_nullable FROM INFORMATION_SCHEMA.COLUMNS WHERE upper(table_name) = 'VARONES'" 
df1 = pd.read_sql_query(text(QUERY), engine)
print(f'El contenido de la tabla VARONES\n')
print(df1)
print()

El contenido de la tabla VARONES

   column_name          data_type  character_maximum_length is_nullable
0   enfermo_no            integer                       NaN          NO
1  inscripcion            integer                       NaN         YES
2     apellido  character varying                      15.0         YES
3    direccion  character varying                      20.0         YES
4    fecha_nac               date                       NaN         YES
5            s  character varying                       1.0         YES
6          nss            integer                       NaN         YES



##### 70. Crear la tabla EMPLEADOS2 con la misma estructura que la tabla EMPLEADOS y conteniendo los datos de oficio PRESIDENTE o comisión mayor que el 25% del salario.

In [22]:
with engine.begin() as connection:
    # Se crea una copia desde la tabla ENFERMO
    QUERY = """
    CREATE TABLE EMPLEADOS2 AS TABLE EMPLEADO WITH NO DATA;
    """
    result = connection.execute(text(QUERY))
    # Se agrega la primary key y si las hubiera, las restricciones
    QUERY = """
    ALTER TABLE EMPLEADOS2 ADD PRIMARY KEY (EMP_NO);
    """
    # Se comprueba que la tabla ha sido creada
    result = connection.execute(text(QUERY))
    QUERY = """
    SELECT * FROM EMPLEADOS2;
    """
    result = connection.execute(text(QUERY))
print(f'La nueva tabla tiene {result.rowcount} fila(s)')


La nueva tabla tiene 0 fila(s)


In [23]:
QUERY = f"SELECT column_name, data_type, character_maximum_length, is_nullable FROM INFORMATION_SCHEMA.COLUMNS WHERE upper(table_name) = 'EMPLEADOS2'" 
df1 = pd.read_sql_query(text(QUERY), engine)
print(f'El contenido de la tabla EMPLEADOS2\n')
print(df1)
print()

El contenido de la tabla EMPLEADOS2

  column_name          data_type  character_maximum_length is_nullable
0      emp_no            integer                       NaN          NO
1    apellido  character varying                      10.0         YES
2      oficio  character varying                      10.0         YES
3         dir            integer                       NaN         YES
4  fecha_alta               date                       NaN         YES
5     salario            integer                       NaN         YES
6    comision            integer                       NaN         YES
7     dept_no            integer                       NaN         YES



In [24]:
with engine.begin() as connection:
    QUERY = """
    INSERT INTO EMPLEADOS2
    SELECT * 
    FROM EMPLEADO
    WHERE (COALESCE(COMISION, 0) > (SALARIO * 0.25)) OR UPPER(OFICIO) = 'PRESIDENTE';
    """
    result = connection.execute(text(QUERY))
print(f' se ha modificado {result.rowcount} fila(s)')

 se ha modificado 3 fila(s)


In [25]:
QUERY = """
SELECT * FROM EMPLEADOS2;
"""
# Ejecutar la consulta y obtener los resultados en un DataFrame
df = pd.read_sql_query(text(QUERY), engine)
df

,emp_no,apellido,oficio,dir,fecha_alta,salario,comision,dept_no
0,7521,Sala,Vendedor,7698.0,1981-02-22,162500,65000.0,30
1,7654,Arenas,Vendedor,7698.0,1981-09-28,162500,182000.0,30
2,7839,Rey,Presidente,NaN,1981-11-17,650000,NaN,10


In [26]:
engine.dispose()

# Autor
### Oscar Gutierrez Leal
18 - 04 - 2024